In [1]:
import pandas as pd 
import numpy as np 
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from keras.models import Sequential
from keras.utils import to_categorical
from keras.layers import Dense, LSTM, BatchNormalization, Flatten
from keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler, TensorBoard
import matplotlib.pyplot as plt
import seaborn as sns

2023-03-02 17:41:03.849011: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-02 17:41:03.990956: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/google/chrome:/opt/google/chrome/lib
2023-03-02 17:41:03.990980: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-02 17:41:04.806695: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: can

In [2]:
df_train = pd.read_csv('Train.csv')
df_test = pd.read_csv('Test.csv')

In [3]:
df_train.head()

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
0,462809,Male,No,22,No,Healthcare,1.0,Low,4.0,Cat_4,D
1,462643,Female,Yes,38,Yes,Engineer,NaN,Average,3.0,Cat_4,A
2,466315,Female,Yes,67,Yes,Engineer,1.0,Low,1.0,Cat_6,B
3,461735,Male,Yes,67,Yes,Lawyer,0.0,High,2.0,Cat_6,B
4,462669,Female,Yes,40,Yes,Entertainment,NaN,High,6.0,Cat_6,A


In [4]:
def df_dados(df):
    
    df_details = {"unique" : df.nunique(),
                        "dtype" : df.dtypes,
                        "null" : df.isna().sum()}
                        
    df_details = pd.DataFrame(df_details)

    display(df_details), print(f"Quantidade de dados duplicados: {df.duplicated().sum()}")

In [5]:
df_dados(df_train)

df_dados(df_test)

,unique,dtype,null
ID,8068,int64,0
Gender,2,object,0
Ever_Married,2,object,140
Age,67,int64,0
Graduated,2,object,78
Profession,9,object,124
Work_Experience,15,float64,829
Spending_Score,3,object,0
Family_Size,9,float64,335
Var_1,7,object,76


Quantidade de dados duplicados: 0


,unique,dtype,null
ID,2627,int64,0
Gender,2,object,0
Ever_Married,2,object,50
Age,67,int64,0
Graduated,2,object,24
Profession,9,object,38
Work_Experience,15,float64,269
Spending_Score,3,object,0
Family_Size,9,float64,113
Var_1,7,object,32


Quantidade de dados duplicados: 0


In [6]:
df_train, df_test = df_train.dropna(axis = 0), df_test.dropna(axis = 0) 

df_train, df_test = df_train.reset_index(drop = True), df_test.reset_index(drop = True)

df_train, df_test = df_train.drop(["ID"], axis= 1), df_test.drop(["ID"], axis= 1)

In [7]:
df_dados(df_train)

df_dados(df_test)

,unique,dtype,null
Gender,2,object,0
Ever_Married,2,object,0
Age,67,int64,0
Graduated,2,object,0
Profession,9,object,0
Work_Experience,15,float64,0
Spending_Score,3,object,0
Family_Size,9,float64,0
Var_1,7,object,0
Segmentation,4,object,0


Quantidade de dados duplicados: 392


,unique,dtype,null
Gender,2,object,0
Ever_Married,2,object,0
Age,67,int64,0
Graduated,2,object,0
Profession,9,object,0
Work_Experience,15,float64,0
Spending_Score,3,object,0
Family_Size,9,float64,0
Var_1,7,object,0
Segmentation,4,object,0


Quantidade de dados duplicados: 20


In [8]:
def alter_data_unique(df):

    label_encoder = preprocessing.LabelEncoder()

    columns_ = df.columns

    for column in columns_:

        if df[column].dtype == 'object':
            df_unique = df[column].unique()
            print(f"{column} - {df_unique}")
            df[column] = label_encoder.fit_transform(df[column])
            df_unique = df[column].unique()
            print(f"{column} - {df_unique}")
            

In [9]:
def normalization_return_x_and_y(df):

    columns_ = df.columns

    for column in columns_:
        if column != "Segmentation":
            df[column] = df[column] / df[column].max()

    df_x = df.drop(["Segmentation"], axis = 1)
    df_y = df["Segmentation"]
    x = np.array(df_x)
    y = to_categorical(df_y, len(df_y.unique()))

    return x, y

In [10]:
alter_data_unique(df_train)

Gender - ['Male' 'Female']
Gender - [1 0]
Ever_Married - ['No' 'Yes']
Ever_Married - [0 1]
Graduated - ['No' 'Yes']
Graduated - [0 1]
Profession - ['Healthcare' 'Engineer' 'Lawyer' 'Artist' 'Doctor' 'Homemaker'
 'Entertainment' 'Marketing' 'Executive']
Profession - [5 2 7 0 1 6 3 8 4]
Spending_Score - ['Low' 'High' 'Average']
Spending_Score - [2 1 0]
Var_1 - ['Cat_4' 'Cat_6' 'Cat_7' 'Cat_3' 'Cat_1' 'Cat_2' 'Cat_5']
Var_1 - [3 5 6 2 0 1 4]
Segmentation - ['D' 'B' 'C' 'A']
Segmentation - [3 1 2 0]


In [11]:
x_train, y_train = normalization_return_x_and_y(df_train)

In [12]:
alter_data_unique(df_test)

Gender - ['Female' 'Male']
Gender - [0 1]
Ever_Married - ['Yes' 'No']
Ever_Married - [1 0]
Graduated - ['Yes' 'No']
Graduated - [1 0]
Profession - ['Engineer' 'Healthcare' 'Executive' 'Doctor' 'Artist' 'Lawyer'
 'Entertainment' 'Homemaker' 'Marketing']
Profession - [2 5 4 1 0 7 3 6 8]
Spending_Score - ['Low' 'Average' 'High']
Spending_Score - [2 0 1]
Var_1 - ['Cat_6' 'Cat_4' 'Cat_3' 'Cat_1' 'Cat_2' 'Cat_5' 'Cat_7']
Var_1 - [5 3 2 0 1 4 6]
Segmentation - ['B' 'A' 'C' 'D']
Segmentation - [1 0 2 3]


In [13]:
x_test, y_test = normalization_return_x_and_y(df_test)

In [14]:
model = Sequential()

model.add(Dense(1000, activation = 'relu'))
model.add(Dense(500, activation = 'relu'))
model.add(Dense(4, activation = 'softmax'))

model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics=['accuracy'])

2023-03-02 17:41:06.837574: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/google/chrome:/opt/google/chrome/lib
2023-03-02 17:41:06.837602: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-03-02 17:41:06.837628: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (pop-os): /proc/driver/nvidia/version does not exist
2023-03-02 17:41:06.837893: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [15]:
# early_stopping = EarlyStopping(
#     monitor = 'loss', 
#     min_delta = 0.05, 
#     patience = 5
# )

# checkpoint = ModelCheckpoint(
#     'model.w', 
#     monitor='loss', 
#     save_best_only=True, 
#     save_weights_only=True
# )

In [16]:
checkpoint = TensorBoard(log_dir="./logs")

In [19]:
def scheduler(epochs, batch_size):
    if epochs < 10:
        return batch_size
    else:
        batch_size += (batch_size * 0.01)
        return batch_size

In [20]:
callback = LearningRateScheduler(scheduler)

In [21]:
model.fit(x_train, y_train, epochs = 100, batch_size = 10, callbacks=[checkpoint, callback])

Epoch 1/100
667/667 [==============================] - 6s 7ms/step - loss: 1.1529 - accuracy: 0.4821 - lr: 0.0010
Epoch 2/100
667/667 [==============================] - 5s 7ms/step - loss: 1.1165 - accuracy: 0.5040 - lr: 0.0010
Epoch 3/100
667/667 [==============================] - 5s 7ms/step - loss: 1.1025 - accuracy: 0.5158 - lr: 0.0010
Epoch 4/100
667/667 [==============================] - 5s 7ms/step - loss: 1.0934 - accuracy: 0.5152 - lr: 0.0010
Epoch 5/100
667/667 [==============================] - 5s 7ms/step - loss: 1.0821 - accuracy: 0.5197 - lr: 0.0010
Epoch 6/100
667/667 [==============================] - 5s 7ms/step - loss: 1.0771 - accuracy: 0.5260 - lr: 0.0010
Epoch 7/100
667/667 [==============================] - 5s 7ms/step - loss: 1.0673 - accuracy: 0.5305 - lr: 0.0010
Epoch 8/100
667/667 [==============================] - 5s 7ms/step - loss: 1.0624 - accuracy: 0.5293 - lr: 0.0010
Epoch 9/100
667/667 [==============================] - 5s 7ms/step - loss: 1.0583 - accu

In [24]:
loss, accuracy = model.evaluate(x_test, y_test)

68/68 [==============================] - 0s 3ms/step - loss: 3.6146 - accuracy: 0.3083


In [23]:
# model.fit(x_train, y_train, epochs= 300, batch_size= 164, callbacks=[checkpoint], verbose = 0)